In [1]:
R.version.string
print("email: 'Maciek Sykulski'<macieksk@gmail.com>")
length(ls())

[1] "R version 3.2.2 (2015-08-14)"

[1] "email: 'Maciek Sykulski'<macieksk@gmail.com>"


[1] 0

In [16]:
projDir<-"./"
projectPrefix<-"R_supervised_analysis_MP_cTCC_Project"

#
load.sources<-function(){
    for (f in  c("utils.R", #Contains inpar, save_new_image, etc
                 "heatmap.clust.R",
                 "Choi.et.al.genes.R",
                 "iterative.unsup.sup.R",
                 "clustcompare.R"
                 #"circos.R"
                 #...  
                 ) 
        ){
    source(paste(projDir,"/src/",f,sep=""))
   }
}
load.sources()

In [18]:
#Load project from .Rdata
system.time(load(paste(projDir,"/r_saved_images/",projectPrefix,".Rdata",sep="")))
lsos(n=30)
gc()

Warning message:
: namespace ‘pROC’ is not available and has been replaced
by .GlobalEnv when processing object ‘roc1’

   user  system elapsed 
 92.500   1.890  94.517 

Loading required package: lumi
Warning message:
In library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, : there is no package called ‘lumi’

ERROR: Error in .requirePackage(package): unable to find required package ‘lumi’


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,11713563.0,625.6,17371378.0,927.8,12931886.0,690.7
Vcells,597949653.0,4562.0,797004094.0,6080.7,664081033.0,5066.6


In [7]:
#Save project .Rdata
gc()
#system.time(save_new_image())

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,329183.0,17.6,592000.0,31.7,550603.0,29.5
Vcells,544411.0,4.2,1023718.0,7.9,785564.0,6.0


In [ ]:
library(cluster)
library(MASS)

#library(lumi)
#library(lumiHumanIDMapping)
#library(lumiHumanAll.db)

#install.packages("./src/mylimma_3.18.13.tar.gz")
# Modyfikowałem pakiet limma, bo był w nim bug, którego zgłosiłem.
# Może już jest poprawiony w nowej wersji?
library(limma)

library(rpca)

library(gplots)
library(png)

#library(multicore)
library(annotate)
library(GOstats)

library(pamr)
library(flare)

library(lattice)
library(multtest)
library(matrixStats)
library(extrafont)
library(enrichvs)
library(statmod)

load.sources()

sessionInfo()